# Batch Data Queries

**These queries were all ran inside of a Databricks Notebook, referencing the clean data from the three DataFrames: `df_geo`, `df_pin`, `df_user`**

#### 1. Finding the most popular Pinterest Category that people post to based on their country

In [ ]:
from pyspark.sql.functions import *

In [ ]:
popular_category = df_pin \
    .join(df_geo, df_pin.ind == df_geo.ind) \
    .groupBy("country", "category") \
    .agg(count("category").alias("category_count")) \
    .orderBy("category_count", ascending=False) \
    .limit(5)

display(popular_category)

country,category,category_count
Algeria,quotes,31
Albania,art,30
Afghanistan,education,15
Andorra,tattoos,12
American Samoa,tattoos,11


#### 2. Find which was the most popular category each year.

In [ ]:
popular_category_per_year = df_pin \
    .join(df_geo, df_pin.ind == df_geo.ind) \
    .groupBy(year("timestamp").alias("post_year"), "category") \
    .agg(count("category").alias("category_count")) \
    .orderBy("category_count", ascending=False) \
    .limit(5)

display(popular_category_per_year)

post_year,category,category_count
2018,christmas,45
2019,art,37
2020,art,35
2019,christmas,32
2021,education,32


#### 3. For each country, find the user with the most followers

In [ ]:
from pyspark.sql.window import Window

window_spec = Window.partitionBy("country")

most_followed_per_country = df_pin \
    .join(df_geo, df_pin.ind == df_geo.ind) \
    .withColumn("MaxFollowingByCountry", max("follower_count").over(window_spec)) \
    .where(col("follower_count") == col("MaxFollowingByCountry")) \
    .select("country", "poster_name", "follower_count").distinct() \
    .orderBy(desc(col("follower_count")))

display(most_followed_per_country)

country,poster_name,follower_count
Angola,Tastemade,8000000
American Samoa,Mamas Uncut,8000000
Azerbaijan,Style Me Pretty,6000000
Burkina Faso,Behance,6000000
Albania,The Minds Journal,5000000
Algeria,Apartment Therapy,5000000
Bouvet Island (Bouvetoya),POPSUGAR,5000000
Comoros,Ruffled,4000000
Guam,imgur,4000000
United States Virgin Islands,"Ugly Duckling House | DIY, Home, Woodworking, Crafts & Recipes",4000000


#### 3-1. Based on the above query, find the country with the user who has the most followers

In [ ]:
country_of_max_followed_user = most_followed_per_country \
    .groupBy("country") \
    .agg(max("follower_count").alias("follower_count")) \
    .orderBy(desc(col("follower_count")), col("country")) \
    .limit(1)

display(country_of_max_followed_user)

country,follower_count
American Samoa,8000000


#### 4. Find the most popular category for different age groups

In [ ]:
# Creating a temporary view for Spark SQL queries.
df_user.join(df_pin, df_pin.ind == df_user.ind).createOrReplaceTempView("pin_user_table")

In [ ]:
age_group_query = spark.sql("""
SELECT category,
    CASE
        WHEN age BETWEEN 18 AND 24 THEN '18-24'
        WHEN age BETWEEN 25 AND 35 THEN '25-35'
        WHEN age BETWEEN 36 AND 50 THEN '36-50'
        WHEN age > 50 THEN '50+'
    END AS age_group
FROM pin_user_table""")

display(age_group_query)

category,age_group
christmas,25-35
diy-and-crafts,18-24
christmas,25-35
christmas,36-50
diy-and-crafts,18-24
diy-and-crafts,18-24
travel,25-35
christmas,36-50
education,25-35
travel,36-50


In [ ]:
window_spec = Window.partitionBy(col("age_group")).orderBy(desc(col("category_count")))

popular_cat_for_ages = age_group_query \
        .groupBy("age_group", "category") \
        .agg(count("category").alias("category_count")) \
        .withColumn("row_number", row_number().over(window_spec)) \
        .where(col("row_number") == 1) \
        .select("age_group", "category", "category_count")

popular_cat_for_ages.show()

+---------+---------+--------------+
age_group| category|category_count|
+---------+---------+--------------+
 18-24| art| 87|
 25-35|christmas| 55|
 36-50| finance| 39|
 50+| vehicles| 20|
+---------+---------+--------------+

#### 5. Find the median follower count for different age groups

In [ ]:
age_group_followers = spark.sql("""
SELECT
    CASE
        WHEN age BETWEEN 18 AND 24 THEN '18-24'
        WHEN age BETWEEN 25 AND 35 THEN '25-35'
        WHEN age BETWEEN 36 AND 50 THEN '36-50'
        WHEN age > 50 THEN '50+'
    END AS age_group,
    percentile_cont(0.5) within GROUP(ORDER BY follower_count) AS median_follower_count
FROM pin_user_table
GROUP BY age_group""")

display(age_group_followers)

age_group,median_follower_count
50+,1000.0
36-50,7000.0
18-24,104000.0
25-35,23000.0


#### 6. Find how many users have joined each year

In [ ]:
users_per_year = df_user \
    .groupBy(year("date_joined").alias("post_year")) \
    .agg(count("username").alias("number_users_joined")) \
    .filter(col("post_year").between(2015, 2020)) \
    .select("post_year", "number_users_joined")

display(users_per_year)

post_year,number_users_joined
2015,676
2016,758
2017,267


#### 7. Find the median follower count of users based on their joining year

In [ ]:
%sql -- Method 1
SELECT EXTRACT(YEAR FROM date_joined) post_year,
        percentile_cont(0.5) within GROUP (ORDER BY follower_count) AS median_follower_count
FROM pin_user_table
GROUP BY EXTRACT(YEAR FROM date_joined)
HAVING EXTRACT(YEAR FROM date_joined) BETWEEN 2015 AND 2020

post_year,median_follower_count
2015,128000.0
2016,19000.0
2017,3000.0


In [ ]:
# Method 2
df_user \
    .join(df_pin, df_user.ind == df_pin.ind) \
    .groupBy(year("date_joined").alias("post_year")) \
    .agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
    .filter(col("post_year").between(2015, 2020)) \
    .show()

+---------+---------------------+
post_year|median_follower_count|
+---------+---------------------+
 2015| 128000|
 2016| 19000|
 2017| 3000|
+---------+---------------------+

#### 8. Find the median follower count of users based on their joining year and age group

In [ ]:
age_group_followers_joindate = spark.sql("""
SELECT 
CASE
    WHEN age BETWEEN 18 AND 24 THEN '18-24'
    WHEN age BETWEEN 25 AND 35 THEN '25-35'
    WHEN age BETWEEN 36 AND 50 THEN '36-50'
    WHEN age > 50 THEN '50+'
END AS age_group,
date_joined, follower_count
FROM pin_user_table""")

# display(age_group_followers_joindate)

In [ ]:
age_group_followers_joindate \
    .groupBy("age_group", year("date_joined").alias("post_year")) \
    .agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
    .orderBy("age_group", "post_year") \
    .show()

+---------+---------+---------------------+
age_group|post_year|median_follower_count|
+---------+---------+---------------------+
 18-24| 2015| 229000|
 18-24| 2016| 46000|
 18-24| 2017| 11000|
 25-35| 2015| 42000|
 25-35| 2016| 22000|
 25-35| 2017| 2000|
 36-50| 2015| 23000|
 36-50| 2016| 8000|
 36-50| 2017| 3000|
 50+| 2015| 14000|
 50+| 2016| 908|
 50+| 2017| 1000|
+---------+---------+---------------------+